In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from prep import get_data
from torch import nn
from kfold_experiments import kfold_normal_training_softgedt, kfold_adversarial_training_softgedt,kfold_normal_training_neural_networks, kfold_adversarial_training_neural_networks
import create_file_folder

# Create dataset

In [2]:
# load dataset
filename = 'wine_red'
create_file_folder(filename)

# Preprocess the data without standardizing
data, ncat,dtypes = get_data(filename)

# create editability vector
non_editable_vector = torch.tensor([1 for i in range(data[:,:-1].shape[1])])

# record datatypes
dtypes[dtypes=='object'] = 'pos_int'
dtypes[dtypes=='float64'] = 'float'

datatypes = dtypes[:-1].to_numpy().astype('object')
datatypes[(datatypes=='int64') & np.min(data[:,:-1]>=0,axis=0)]='pos_int'
datatypes[(datatypes=='float64') & np.min(data[:,:-1]>=0,axis=0)]='pos_float'
datatypes = datatypes.tolist()

# Normal Training

In [3]:
kfold_normal_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight = True)

experiment: 0
Fold: 0
epoch: 0
train acc: tensor(0.8311) valid acc: tensor(0.6125) trigger times: 0
epoch: 1
train acc: tensor(0.8624) valid acc: tensor(0.6250) trigger times: 0
epoch: 2
train acc: tensor(0.8697) valid acc: tensor(0.6156) trigger times: 0
epoch: 3
train acc: tensor(0.8717) valid acc: tensor(0.6125) trigger times: 1
epoch: 4
train acc: tensor(0.8780) valid acc: tensor(0.6250) trigger times: 2
epoch: 5
train acc: tensor(0.8780) valid acc: tensor(0.6187) trigger times: 0
epoch: 6
train acc: tensor(0.8446) valid acc: tensor(0.4344) trigger times: 1
epoch: 7
train acc: tensor(0.4234) valid acc: tensor(0.4344) trigger times: 2
epoch: 8
train acc: tensor(0.4234) valid acc: tensor(0.4344) trigger times: 3
Early Stopping
SoftGeDT Accuracy on Training set: tensor(0.4234)
SoftGeDT Accuracy on Validation set: tensor(0.4344)
SoftGeDT Accuracy on Test set: tensor(0.4250)
Average Accuracy on Test set: tensor(0.4250)
Original Accuracy on Test set: tensor(0.6125)
Average Original Accur

# Adversarial Training

In [16]:
kfold_adversarial_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight=True)

Fold: 0
epoch: 0
train error: 0.47758081334723673 train loss: 3.6082664306767915 validation error 0.68125
epoch: 1
train error: 0.47340980187695514 train loss: 1.5366712755157503 validation error 0.659375
epoch: 2
train error: 0.4348279457768509 train loss: 1.2893322953591133 validation error 0.640625
epoch: 3
train error: 0.40667361835245047 train loss: 1.2701409213610064 validation error 0.628125
epoch: 4
train error: 0.3983315954118874 train loss: 1.2567052672130596 validation error 0.6125
epoch: 5
train error: 0.397288842544317 train loss: 1.2394995744086654 validation error 0.59375
epoch: 6
train error: 0.40041710114702816 train loss: 1.218395692588639 validation error 0.596875
epoch: 7
train error: 0.3920750782064651 train loss: 1.1623373538783992 validation error 0.575
epoch: 8
train error: 0.3816475495307612 train loss: 1.1098850556533701 validation error 0.5875
epoch: 9
train error: 0.3931178310740355 train loss: 1.0933100687450612 validation error 0.578125
FNN Adversarial Err

# Simple Neural Network

In [3]:
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32).double()
        self.layer2 = nn.Linear(32, 16).double()
        self.layer3 = nn.Linear(16, 6).double()
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

## 5-fold cross validation normal training

In [11]:
kfold_normal_training_neural_networks(nn, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train acc: tensor(0.5151) valid acc: tensor(0.5219) trigger times: 0
epoch: 1
train acc: tensor(0.5839) valid acc: tensor(0.5469) trigger times: 0
epoch: 2
train acc: tensor(0.6246) valid acc: tensor(0.5094) trigger times: 0
epoch: 3
train acc: tensor(0.6184) valid acc: tensor(0.6031) trigger times: 1
epoch: 4
train acc: tensor(0.6288) valid acc: tensor(0.6156) trigger times: 0
epoch: 5
train acc: tensor(0.6340) valid acc: tensor(0.6094) trigger times: 0
epoch: 6
train acc: tensor(0.6486) valid acc: tensor(0.5500) trigger times: 1
epoch: 7
train acc: tensor(0.6413) valid acc: tensor(0.6031) trigger times: 2
epoch: 8
train acc: tensor(0.6486) valid acc: tensor(0.5781) trigger times: 0
epoch: 9
train acc: tensor(0.6663) valid acc: tensor(0.6156) trigger times: 1
epoch: 10
train acc: tensor(0.6611) valid acc: tensor(0.5688) trigger times: 0
epoch: 11
train acc: tensor(0.6788) valid acc: tensor(0.5469) trigger times: 1
epoch: 12
train acc: tensor(0.6694) valid acc: tensor(

## 5 fold cross validation adversarial training

In [12]:
kfold_adversarial_training_neural_networks(Net, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train error: 0.5547445255474452 train loss: 1.3409181383784545 validation error 0.565625
epoch: 1
train error: 0.4942648592283629 train loss: 1.1218382638428037 validation error 0.50625
epoch: 2
train error: 0.4848800834202294 train loss: 1.0738485623202387 validation error 0.525
epoch: 3
train error: 0.4608967674661105 train loss: 1.0364015899492676 validation error 0.484375
epoch: 4
train error: 0.470281543274244 train loss: 1.0257578161877206 validation error 0.5
epoch: 5
train error: 0.470281543274244 train loss: 1.0104146035273667 validation error 0.509375
epoch: 6
train error: 0.44629822732012514 train loss: 1.0130856415698846 validation error 0.50625
epoch: 7
train error: 0.45776850886339937 train loss: 0.9949077310173624 validation error 0.49375
epoch: 8
train error: 0.4504692387904067 train loss: 0.98111490668658 validation error 0.5
epoch: 9
train error: 0.43899895724713245 train loss: 0.9896449705323179 validation error 0.5
epoch: 10
train error: 0.446298227